<a href="https://colab.research.google.com/github/con11235/covid19_visualize/blob/master/Data%20Processing/%EC%84%9C%EC%9A%B8%EC%A7%80%EC%97%AD%EB%B3%84%ED%99%95%EC%A7%84%EC%9E%90%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina' 
!apt -qq -y install fonts-nanum
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

import datetime
import plotly.express as px

import plotly
from plotly.graph_objs import Scatter, Line, Layout
 
import importlib
import json
 
from plotly.subplots import make_subplots
import plotly.graph_objects as go


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [3]:
patientinfo=pd.read_csv('/content/gdrive/My Drive/dacon/COVID_19/PatientInfo.csv')
patientinfo.head(3)

,patient_id,sex,age,country,province,city,infection_case,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,male,50s,Korea,서울,강서구,overseas inflow,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,male,30s,Korea,서울,중랑구,overseas inflow,NaN,31,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,male,50s,Korea,서울,종로구,contact with patient,2002000001,17,NaN,2020-01-30,2020-02-19,NaN,released


### 서울 확진자만 분류

In [4]:
patientinfo_seoul=patientinfo[patientinfo['province']=='서울']

In [5]:
patient_seoul=patientinfo_seoul[['province','city','confirmed_date']]
patient_seoul['total_ads']=patient_seoul['province']+' '+patientinfo_seoul['city']

지역별로 분류

In [6]:
patient_seoul=patient_seoul[['confirmed_date','total_ads']]
patient_seoul['cnt']=1
patient_seoul.head(3)

,confirmed_date,total_ads,cnt
0,2020-01-23,서울 강서구,1
1,2020-01-30,서울 중랑구,1
2,2020-01-30,서울 종로구,1


혹시 몰라서 날짜순으로 배열

In [7]:
patient_seoul=patient_seoul.sort_values(by=['confirmed_date',],ascending=True)

날짜별 그 지역 확진자 수로 병합

ex) 1/23 종로구1, 종로구1 => 종로구2

In [8]:
patient_seoul=patient_seoul.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()
patient_seoul.head(3)

,confirmed_date,total_ads,cnt
0,2020-01-23,서울 강서구,1
1,2020-01-30,서울 마포구,1
2,2020-01-30,서울 종로구,1


지역별 누적 확진자 추가

In [10]:
patient_seoul['cnt_sum']=patient_seoul.groupby('total_ads')['cnt'].cumsum()
patient_seoul.head(3)

,confirmed_date,total_ads,cnt,cnt_sum
0,2020-01-23,서울 강서구,1,1
1,2020-01-30,서울 마포구,1,1
2,2020-01-30,서울 종로구,1,1


In [13]:
fig=px.line(patient_seoul,x='confirmed_date',y='cnt_sum',color='total_ads')
fig.update_layout(
    title={
        'text': "지역별 확진자 변화(서울 한정)",
        'y':1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

### 1월/2월초/2월말/3월초/3월말/4월초/4월말/5월초/5월말/6월초 나누기

In [18]:
is_date=patient_seoul['confirmed_date']<'2020-02-01'
patient_seoul_jan=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)

patient_seoul_jan['confirmed_date']='1월 말'
patient_seoul_jan=patient_seoul_jan.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()
patient_seoul_jan

,confirmed_date,total_ads,cnt
0,1월 말,서울 강서구,1
1,1월 말,서울 마포구,1
2,1월 말,서울 성북구,1
3,1월 말,서울 종로구,3
4,1월 말,서울 중랑구,1


In [20]:
#2월 초
is_date_1=patient_seoul_1['confirmed_date']<'2020-02-15'
patient_seoul_feb_1=patient_seoul[is_date_1]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_feb_1['confirmed_date']='2월 초'
patient_seoul_feb_1=patient_seoul_feb_1.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#2월 말
is_date=patient_seoul['confirmed_date']<'2020-03-01'
patient_seoul_feb_2=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_feb_2['confirmed_date']='2월 말'
patient_seoul_feb_2=patient_seoul_feb_2.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#3월 초
is_date=patient_seoul['confirmed_date']<'2020-03-16'
patient_seoul_mar_1=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_mar_1['confirmed_date']='3월 초'
patient_seoul_mar_1=patient_seoul_mar_1.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#3월 말
is_date=patient_seoul['confirmed_date']<'2020-04-01'
patient_seoul_mar_2=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_mar_2['confirmed_date']='3월 말'
patient_seoul_mar_2=patient_seoul_mar_2.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#4월 초

is_date=patient_seoul['confirmed_date']<'2020-04-16'
patient_seoul_apr_1=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_apr_1['confirmed_date']='4월 초'
patient_seoul_apr_1=patient_seoul_apr_1.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#4월 말
is_date=patient_seoul['confirmed_date']<'2020-05-01'
patient_seoul_apr_2=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_apr_2['confirmed_date']='4월 말'
patient_seoul_apr_2=patient_seoul_apr_2.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()



5월초/말, 6월초/말

In [23]:
#5월 초
is_date=patient_seoul['confirmed_date']<'2020-05-16'
patient_seoul_may_1=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_may_1['confirmed_date']='5월 초'
patient_seoul_may_1=patient_seoul_may_1.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#5월 말
is_date=patient_seoul['confirmed_date']<'2020-06-01'
patient_seoul_may_2=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_may_2['confirmed_date']='3월 말'
patient_seoul_may_2=patient_seoul_may_2.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#6월 초

is_date=patient_seoul['confirmed_date']<'2020-06-16'
patient_seoul_june_1=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_june_1['confirmed_date']='6월 초'
patient_seoul_june_1=patient_seoul_june_1.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#6월 말
is_date=patient_seoul['confirmed_date']<'2020-07-01'
patient_seoul_june_2=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_june_2['confirmed_date']='6월 말'
patient_seoul_june_2=patient_seoul_june_2.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()


In [25]:
patient_seoul_ads=pd.concat([patient_seoul_jan,patient_seoul_feb_1,patient_seoul_feb_2,patient_seoul_mar_1,patient_seoul_mar_2,
                             patient_seoul_apr_1,patient_seoul_apr_2,patient_seoul_may_1,patient_seoul_may_2,
                             patient_seoul_june_1,patient_seoul_june_2],axis=0)

In [27]:
patient_seoul_ads['cnt_sum']=patient_seoul_ads.groupby('total_ads')['cnt'].cumsum()

In [28]:
fig=px.line(patient_seoul_ads,x='confirmed_date',y='cnt_sum',color='total_ads')
fig.update_layout(
    title={
        'text': "지역별 확진자 변화(서울 한정)",
        'y':1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()